In [ ]:
import string
import random
import timeit

class Decoder:
    def __init__(self, globalText, encodedText, keyLength):
        self.myDict = set()
        self.length = keyLength
        self.myKeys = []
        self.keysRates = []
        self.sortedKeysByRate = []
        self.myTxt = encodedText.lower()
        self.destIdx = 0
        self.myTxtSize = len(encodedText)
        self.encTxt = encodedText
        self.endingState = self.countMeaningfulWords(encodedText)
        lowerGlobalTxt = globalText.lower()
        self.myDict = self.createDict(lowerGlobalTxt)
        self.myDict.remove('')

    def countMeaningfulWords(self, text):
        counterWords = 0
        myWord = ""
        for c in text:
            if ((ord(c) >= 97 and ord(c) <= 122) or (ord(c) >= 65 and ord(c) <= 90)):
                myWord += c
            else:
                if(myWord != ""):
                    counterWords +=1
                myWord = ""

        return counterWords
    def decodeFinal(self, key):
        idx = -1
        decodedTxt = ""
        for i in range(self.myTxtSize):
            if(ord(self.myTxt[i]) >= 97 and ord(self.myTxt[i]) <= 122):
                idx += 1

                if(ord(self.myTxt[i]) - ord(key[idx%14]) < 0):
                    c = chr(ord(self.myTxt[i]) - ord(key[idx%14]) + 123)
                    if(ord(self.encTxt[i]) >= 65 and ord(self.encTxt[i]) <= 90):
                        c = c.upper()
                    decodedTxt += c
                else:
                    c = chr(ord(self.myTxt[i]) - ord(key[idx%14]) + 97)
                    if(ord(self.encTxt[i]) >= 65 and ord(self.encTxt[i]) <= 90):
                        c = c.upper()
                    decodedTxt += c
            else:
                decodedTxt += self.myTxt[i]

        return decodedTxt

    def createDict(self, text):
        cDict = set()
        myWord = ""
        for c in text:
            if ord(c) >= 97 and ord(c) <= 122:
                myWord += c
            else:
                cDict.add(myWord)
                myWord = ""

        return cDict

    def createRandomKeys(self):
        for i in range(100):
            res = ''.join(random.choices(string.ascii_uppercase, k = self.length))
            self.myKeys.append(res.lower())

    def decodeWithKey(self, key):
        idx = -1
        decodedTxt = ""
        for i in range(self.myTxtSize):
            if(ord(self.myTxt[i]) >= 97 and ord(self.myTxt[i]) <= 122):
                idx += 1

                if(ord(self.myTxt[i]) - ord(key[idx%14]) < 0):
                    decodedTxt += chr(ord(self.myTxt[i]) - ord(key[idx%14]) + 123)
                else:
                    decodedTxt += chr(ord(self.myTxt[i]) - ord(key[idx%14]) + 97)
            else:
                decodedTxt += self.myTxt[i]

        return decodedTxt

    def rateKey(self, key):
        decodedTxt = self.decodeWithKey(key)

        counter = 0
        myWord = ""
        for c in decodedTxt:
            if ord(c) >= 97 and ord(c) <= 122:
                myWord += c
            else:
                if(myWord in self.myDict and myWord != ""):
                    counter += 1
                myWord = ""

        return counter

    def mutation(self, key, sortedKeysByRate2, scoreKey, identicalCheck):
        myList = [1,2,3,4,5,6,7,8,9,10]
        myKey = ""

        for i in range(14):
            randIdx = (random.choice(myList) + random.choice(myList)) % 14
            if(randIdx > 10):
                randChar = random.choice(string.ascii_letters)
                myKey += randChar.lower()
                currKey = myKey + key[i+1:]
                scoreMyKey = self.rateKey(currKey)

                if(scoreMyKey > scoreKey):
                    key = currKey
                    scoreKey = scoreMyKey
            else:
                myKey += key[i]


        sortedKeysByRate2.append((key, scoreKey))


    def crossOver(self, key1, key2, crossOverKeys):
        myList = [1,2,3,4,5,6,7,8,9,10]
        randIdx1 = (random.choice(myList) + random.choice(myList)) % 14
        minn = randIdx1
        child1 = key1[:minn] + key2[minn:]
        child2 = key2[:minn] + key1[minn:]
        child1Score = self.rateKey(child1)
        child2Score = self.rateKey(child2)
        crossOverKeys.append((child1, child1Score))
        crossOverKeys.append((child2, child2Score))

    def optimization(self):
        key = self.sortedKeysByRate[0][0]
        scoreKey = self.sortedKeysByRate[0][1]
        myKey =""
        for i in range(self.destIdx,14):
            for randChar in string.ascii_letters:
                strLst = list(key)
                strLst[i] = randChar.lower()
                currKey = "".join(strLst)
                scoreMyKey = self.rateKey(currKey)


                if(scoreMyKey > scoreKey):
                    self.destIdx = i
                    return (currKey, scoreMyKey)

    def decode(self):
        self.createRandomKeys()
        for key in self.myKeys:
            score = self.rateKey(key)
            self.keysRates.append((key, score))

        self.sortedKeysByRate = sorted(self.keysRates, key = lambda x: x[1])
        while(1):
            sortedKeysByRate2 = []
            identicalCheck = set()
            for key in self.sortedKeysByRate:
                self.mutation(key[0], sortedKeysByRate2, key[1], identicalCheck)

            self.sortedKeysByRate = sortedKeysByRate2
            oddCounter = -1
            crossOverKeys = []
            for i in range(len(self.sortedKeysByRate)-1, 0 , -1):
                oddCounter += 1
                if(oddCounter % 2 == 0):
                    key = self.sortedKeysByRate[i]
                    nextKey = self.sortedKeysByRate[i-1]
                    self.crossOver(key[0], nextKey[0], crossOverKeys)

                if(oddCounter == int(len(self.sortedKeysByRate)/2) - 2):
                    break

            sortedCrossOversByRate = sorted(crossOverKeys, key = lambda x: x[1])
            sortedKeysByRate3 = []
            counter = 1
            self.sortedKeysByRate = sorted(sortedKeysByRate2, key = lambda x: x[1])
            sortedCrossOversByRate.extend(self.sortedKeysByRate[50:])
            self.sortedKeysByRate = sortedCrossOversByRate
            if(self.sortedKeysByRate[-1][1] == self.endingState):
                return self.decodeFinal(self.sortedKeysByRate[-1][0])

encodedText = open('encoded_test.txt').read()
globalText = open('global_text.txt').read()
d = Decoder(globalText, encodedText, keyLength=14)
decodedText = d.decode()
print(decodedText)

Old Grove Rd.
Nassau Point
Peconic, Long Island

August 2nd, 1939

F.D. Roosevelt,
President of the United States,
White House
Washington, D.C.

Sir:

Some recent work by E. Fermi and L. Szilard, which has been communicated to me in manuscript, leads me to expect that the element uranium may be turned into a new and important source of energy in the immediate future. Certain aspects of the situation which has arisen seem to call for watchfulness and, if necessary, quick action on the part of the Administration. I believe therefore that it is my duty to bring to your attention the following facts and recommendations:

In the course of the last four months it has been made probable—through the work of Joliot in France as well as Fermi and Szilard in America—that it may become possible to set up a nuclear chain reaction in a large mass of uranium by which vast amounts of power and large quantities of new radium-like elements would be generated. Now it appears almost certain that this coul